# Pacotes que precisam ser instalados:
Comandos:
```shell
sudo pip install selenium    # Automatiza o navegador
sudo pip install bs4         # Html
sudo pip install html5lib    # 'parser' necessário para o bs4
```
<details>
    <summary> Comandos para serem instalados (Todos os pacotes de uma vez)</summary>
    <p><b>Instalação:</b><br><i>
       <b>sudo</b> pip install bs4 html5lib textract</i>
    </p>
    <p><b>Remoção:</b><br><i>
        <b>sudo</b> pip uninstall bs4 html5lib textract</i>
    </p>
</details>

### Configurações:
* Navegador Chrome -> versão 74.0.3729.108 (Official Build) (64-bit)
* Chrome webdriver -> versão 74.0.3729.6

In [1]:
%autosave 120

Autosaving every 120 seconds


# Imports

In [2]:
from selenium import webdriver # automatiza nova guia do navegador em plano de fundo
from bs4 import BeautifulSoup  # necessário para trabalhar com o html
from selenium.webdriver.common.keys import Keys # Tecla ENTER

import re as regex # expressao regular para strings (limpeza de \t\n)
from os.path import abspath # diretorio atual

from packages.SScraping import BasicXml # classe criada para gerar o XML

# URL's

In [3]:
url_login      = "https://sigaa.unifei.edu.br/sigaa/verTelaLogin.do"
url_componente = "https://sigaa.unifei.edu.br/sigaa/geral/componente_curricular/busca_geral.jsf"
url_grade      = "https://sigaa.unifei.edu.br/sigaa/geral/estrutura_curricular/busca_geral.jsf"

# Código e sigla do curso

**Nota**
> Perceba que para a **SIGLA** não há um padrão definido, i.e. ECO/EEL/ECA em contrapartida EMT


| Cód     	| Ano-Período 	| Matriz Curricular                                                	|
|---------	|-------------	|------------------------------------------------------------------	|
| 0202015 	| 2015.1      	| ENGENHARIA ELÉTRICA - Presencial - MTN - Semestral - BACHARELADO 	|
| 0202012 	| 2012.1      	| ENGENHARIA ELÉTRICA - Presencial - MTN - Semestral - BACHARELADO 	|
| 0202010 	| 2010.1      	| ENGENHARIA ELÉTRICA - Presencial - MTN - Semestral - BACHARELADO 	|

---
| Cód     	| Ano-Período 	| Matriz Curricular                                                     	|
|---------	|-------------	|-----------------------------------------------------------------------	|
| 0192015 	| 2015.1      	| ENGENHARIA DE COMPUTAÇÃO - Presencial - MTN - Semestral - BACHARELADO 	|
| 0192012 	| 2012.1      	| ENGENHARIA DE COMPUTAÇÃO - Presencial - MTN - Semestral - BACHARELADO 	|
| 0192010 	| 2010.1      	| ENGENHARIA DE COMPUTAÇÃO - Presencial - MTN - Semestral - BACHARELADO 	|

---
| Cód     	| Ano-Período 	| Matriz Curricular                                                               |
|---------	|-------------	|---------------------------------------------------------------------------------|
| 0702015 	| 2015.1      	| ENGENHARIA DE CONTROLE E AUTOMAÇÃO - Presencial - MTN - Semestral - BACHARELADO |
| 0702012 	| 2012.1      	| ENGENHARIA DE CONTROLE E AUTOMAÇÃO - Presencial - MTN - Semestral - BACHARELADO |
| 0702010 	| 2010.1      	| ENGENHARIA DE CONTROLE E AUTOMAÇÃO - Presencial - MTN - Semestral - BACHARELADO |

<br>
<br>
<br>


In [4]:
codcurso = input("Digite o código do curso.\nDefault: 0192015\tCódigo: ") or '0192015' # Caso não digite, retorna ECO-2015.1 por padrão

Digite o código do curso.
Default: 0192015	Código: 


# Configurando o ambiente automatizado

#### O arquivo do webdriver pode ser encontrado em: [webdriverChrome](http://chromedriver.chromium.org/downloads)

In [5]:
pasta_projeto = abspath('')

In [6]:
chrome_options = webdriver.ChromeOptions() # variavel de opcoes
# chrome_options.add_argument('--headless') # tire o comentário para ocultar a janela
chrome_options.binary_location = "/usr/bin/google-chrome" # caminho do executavel do chrome
chrome_webdriver_path = pasta_projeto + '/webdriver/chromedriver' # caminho do arquivo chromedriver

chrome_webdriver = webdriver.Chrome(executable_path=chrome_webdriver_path, options=chrome_options) # janela

# Fazendo login no _SIGAA_

In [7]:
name = '12638891665' # Pede o login do sigaa (Usamos a tela vista por um aluno)
pwd  = 'Sigaa*0223'

In [8]:
try:
    chrome_webdriver.get(url_login) # acessando url

    user_name = chrome_webdriver.find_element_by_name("user.login") # encontra elemento
    user_name.clear() # limpa o campo
    user_name.send_keys(name) # envia o conteudo de name para o campo

    user_pwd = chrome_webdriver.find_element_by_name("user.senha")
    user_pwd.clear()
    user_pwd.send_keys(pwd)

    user_pwd.send_keys(Keys.ENTER) # tecla enter

except:
    print ('Não foi possível encontrar os elementos na página -> login.\nVerifique sua conexão')

# Buscando grade no Sigaa

##### Montando grade

In [9]:
try:
    chrome_webdriver.get(url_grade)

    chrome_webdriver.find_element_by_id('busca:codigo').clear()
    chrome_webdriver.find_element_by_id('busca:checkCodigo').click()

    box = chrome_webdriver.find_element_by_id('busca:codigo')
    box.send_keys(codcurso)
    box.send_keys(Keys.ENTER)

    chrome_webdriver.find_element_by_id('resultado:relatorio').click()

    soup = BeautifulSoup(chrome_webdriver.page_source, 'html5lib')

except:
    print ('Não foi possível encontrar os elementos na página -> Busca grade.\nVerifique sua conexão')

##### Montando dicionário

In [10]:
ano = (soup.select('#relatorio > table > tbody > tr:nth-of-type(5) > td')[0].text)
ano = ano.replace(' ','')

curso = (soup.select('#relatorio > table > tbody > tr:nth-of-type(2) > td')[0].text)
curso = curso[:curso.find(' - ')]

In [11]:
cname_cod = []
cname_p = []
cname_n = []

In [12]:
for i in soup.find_all('tr', class_='componentes'):
    cod=i.find_all('td')[0].text
    p=i.find_previous('tr', class_='tituloRelatorio').text
    n=i.find_all('td')[1].text
    n=regex.sub('[\t\n]','',n)
    p=regex.sub('[\t\n]','',p)
    n=n[:n.rfind(' - ')]
    aux=regex.search('[\d]+',p)
    
    p = 'Optativa'
    if (aux is not None):
        p=aux.group()
    
    cname_cod.append(cod)
    cname_n.append(n)
    cname_p.append(p)

In [13]:
for i,j,k in zip(cname_cod,cname_n,cname_p):
    print (i,k,j)

BAC021 Optativa MATEMÁTICA III
CEI038 Optativa SEMINÁRIOS DE INTERNACIONALIZAÇÃO – PAÍSES ANGLOFÔNICOS
CEI039 Optativa SEMINÁRIOS DE INTERNACIONALIZAÇÃO – FRANÇA
ECA020 Optativa Instrumentação Industrial II
ECA024 Optativa Robótica
ECAI08 Optativa MEDIDAS ELÉTRICAS E INSTRUMENTAÇÃO
ECO019 Optativa Compiladores
ECO038 Optativa MARATONA DE PROGRAMAÇÃO
ECO041 Optativa Tópicos Especiais em Engenharia de Software
ECO042 Optativa Tópicos Especiais em Microeletrônica
ECO043 Optativa Visão Computacional
ECO044 Optativa PROGRAMAÇÃO PARA DISPOSITIVOS MÓVEIS
ECO045 Optativa Mineração e Armazenamento de Dados
ECO046 Optativa Recuperação de Informação
ECO047 Optativa Tópicos Especiais em Engenharia de Computação
ECO049 Optativa Tópicos Especiais em Matemática
ECO050 Optativa Tópicos Especiais em Física
ECOI27 Optativa NATURAL COMPUTING
ECOI28 Optativa Maratona de Programação I
ECOI29 Optativa MARATONA DE PROGRAMAÇÃO II
ECOI30 Optativa MARATONA DE PROGRAMAÇÃO III
ECOI31 Optativa Maratona de Programa

# Busca: Pré,Parcial e Co Requisito

In [14]:
cname_pre = {}
cname_co = {}

In [15]:
for i,j in zip(cname_cod,cname_p):
    # Dessa forma NÃO procura pre e co para
    # materias optativas
    if j == "Optativa":
        continue

    chrome_webdriver.get(url_componente)
    chrome_webdriver.find_element_by_name('formBusca:checkCodigo').click()

    box = chrome_webdriver.find_element_by_name('formBusca:j_id_jsp_433775776_860')
    box.clear()
    box.send_keys(i)
    box.send_keys(Keys.ENTER) # shows 'listagem' 
    
    # Bloco
    box = chrome_webdriver.find_elements_by_xpath('//a[@title=\'Relatório para impressão\' ]')
    if len(box) == 3:
        box[1].click()
    elif len(box) == 1:
        box[0].click()


    pre = (chrome_webdriver.find_element_by_xpath('//th[text()[contains(.,\'Pré-Requisitos:\')]]/following-sibling::td')).text
    co = (chrome_webdriver.find_element_by_xpath('//th[text()[contains(.,\'Co-Requisitos:\')]]/following-sibling::td')).text

    pre = regex.findall('[\w\d]{3,}',pre) # Tamanho minimo é a palavra OU
    co = regex.findall('[\w\d]{3,}',co)

    cname_pre[i] = pre
    cname_co[i] = co


# Fechando o navegador

In [16]:
chrome_webdriver.close()

In [17]:
for cod,nome in zip(cname_cod,cname_n):
    print (cod,nome, cname_pre[cod], cname_co[cod])

KeyError: 'BAC021'

# Criando estrutura do arquivo XML

In [20]:
to_xml = BasicXml()
to_xml.create_tag('Grade','')
to_xml.create_tag('Curso',curso)
to_xml.create_tag('Ano',ano)
to_xml.create_tag('NDisciplinas', str(len(cname_cod))) # considera todas as disciplinas oferecidas

to_xml.create_tag('Disciplinas','')
for cod,nome,per in zip(cname_cod,cname_n,cname_p):
    to_xml.create_tag('Disciplina', '')
    
    to_xml.create_tag('Nome',nome)
    to_xml.create_tag('Sigla',cod)
    to_xml.create_tag('Periodo', per)
    
    if cod in cname_pre and cname_pre[cod]:
        to_xml.create_tag('PreRequisitoTotal','')
        for j in cname_pre[cod]:
            to_xml.create_tag('Sigla',j)   
        to_xml.end_tag()
    
#     if len(codes_names[i]['parciais']) > 0:
#         to_xml.create_tag('PreRequisitoParcial','')
#         for j in codes_names[i]['parciais']:
#             to_xml.create_tag('Sigla',j)   
#         to_xml.end_tag()
        
    if cod in cname_co and cname_co[cod]:
        to_xml.create_tag('CoRequisito','')
        for j in cname_co[cod]:
            to_xml.create_tag('Sigla',j)   
        to_xml.end_tag()
  
    to_xml.end_tag() # Fim da tag disciplina
to_xml.end_tag() # fim da tag disciplinaS
to_xml.end_tag() # fim da tag Grade

# Gerando o arquivo XML

In [21]:
with open(pasta_projeto + '/xml_files/{}_{} - {}.xml'.format(codcurso,curso,ano), 'w', encoding='utf-8') as file:
    file.write(to_xml.getXml())